# Introduction

This notebook shows how to build ranking model from scratch.
We will train KNRM model on Quora question pairs dataset.

In [ ]:
# Optional TODO: add some explanation, why it is required to have separate models for ranking

In [ ]:
# TODO: read, how to execute notebooks on colab and implement it here

# Install dependencies

In [ ]:
# TODO: find out, what dependencies are required on some empty dataset
#       and install them.
#       consider pip installation rather than conda

In [2]:
! pip install numpy==1.19.2 pandas torch==1.7.1

  Using cached numpy-1.19.2-cp37-cp37m-macosx_10_9_x86_64.whl (15.3 MB)
     |████████████████████████████████| 108.8 MB 1.9 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyro-ppl 1.8.1 requires torch>=1.11.0, but you have torch 1.7.1 which is incompatible.


# Data loading and preparation

In [3]:
! mkdir -p resources && \
  wget https://dl.fbaipublicfiles.com/glue/data/QQP-clean.zip -O - | tar xz -C resources QQP/train.tsv QQP/dev.tsv 

--2023-03-12 11:56:17--  https://dl.fbaipublicfiles.com/glue/data/QQP-clean.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41696084 (40M) [application/zip]
Saving to: ‘STDOUT’

-                   100%[===================>]  39.76M  2.69MB/s    in 13s     

2023-03-12 11:56:32 (3.06 MB/s) - written to stdout [41696084/41696084]



## Show the original dataset

In [4]:
import pandas as pd

In [5]:
quora_dir = './resources/QQP/'

col_names = ['id', 'id_left', 'id_right', 'text_left', 'text_right', 'label']
train_df = pd.read_csv(f"{quora_dir}/train.tsv", sep='\t', names=col_names, skiprows=1)

test_df = pd.read_csv(f"{quora_dir}/dev.tsv", sep='\t', names=col_names, skiprows=1)
train_df

,id,id_left,id_right,text_left,text_right,label
0,133273,213221,213222,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0
1,402555,536040,536041,How do I control my horny emotions?,How do you control your horniness?,1
2,360472,364011,490273,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0
3,150662,155721,7256,What can one do after MBBS?,What do i do after my MBBS ?,1
4,183004,279958,279959,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0
...,...,...,...,...,...,...
363841,57443,100941,83372,How do I make money flying my drone?,How can I use a dji phantom to make money,1
363842,278260,62873,34460,What can you do with an economics degree?,What jobs can you get with an economics degree?,1
363843,136211,217377,217378,What type of current does a battery produce?,How does a generator work and produce current?,0
363844,302720,425744,285638,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0


In [7]:
import aux
vocabulary_list = aux.build_vocabulary(train_df)
print("Vocabulary len:", len(vocabulary_list))
vocabulary_list[:10]

Vocabulary len: 82459


['PAD', 'OOV', 'the', 'what', 'is', 'a', 'i', 'to', 'in', 'how']

# Embeddings

In [ ]:
import aux

In [ ]:
! mkdir -p resources && \
  wget http://nlp.stanford.edu/data/glove.6B.zip -O - | tar xz -C resources glove.6B.50d.txt 

--2023-03-12 11:58:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-03-12 11:58:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-03-12 11:58:58--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘STDOUT’

-         

In [ ]:
# downloaded from http://nlp.stanford.edu/data/glove.6B.zip
# You need file glove.6B.50d.txt in archive
glove_path = './resources/glove.6B.50d.txt'

In [121]:
embeddings_matrix = aux.create_word_embeddings(glove_path, vocabulary_list)
print(embeddings_matrix.shape)
embeddings_matrix

(82461, 50)


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.15411617e-03, -3.69327915e-02,  5.47250399e-03, ...,
        -1.20385573e-01, -5.43640669e-02, -5.84888117e-02],
       [ 1.68536278e-01, -1.09200736e-02, -1.82572998e-01, ...,
         1.73878475e-01, -4.96611106e-02,  1.94339913e-02],
       ...,
       [-2.03040000e-01, -4.22670000e-01, -8.96360000e-01, ...,
         1.03930000e+00, -1.17430000e-01, -6.07190000e-01],
       [-1.52386695e-01,  1.07629474e-01, -1.92355802e-01, ...,
         6.34954164e-05,  1.74816373e-01,  1.22215164e-01],
       [ 1.11420000e-01,  5.47650000e-01,  4.57640000e-01, ...,
        -9.50530000e-01, -6.66570000e-01,  7.22480000e-01]])

# KNRM model

In [95]:
#TODO: write some words about it. Why and how does it works

In [129]:
from knrm import KNRM

model = KNRM(embeddings_matrix,
             freeze_embeddings=True,
             out_layers=[10,5],
             kernel_num=21)

KNRM embeddings is created
KNRM kernels is created
KNRM mlp is created


# Datasets preparation

In [106]:
import dataset
import importlib
import aux
importlib.reload(aux)
importlib.reload(dataset)

<module 'dataset' from '/Users/pavel/src/other/search-ranking/training/dataset.py'>

In [107]:
%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [109]:
class BufferedRandomValuesGenerator:
    def __init__(self, buffer_size):
        self.offset = 0
        self.buffer_size = buffer_size
        self.random_values_buffer = np.random.uniform(0, 1, size=buffer_size)

    def uniform(self):
        value = self.random_values_buffer[self.offset]
        self.offset = (self.offset + 1) % self.buffer_size
        return value

    def choice(self, lst):
        uniform_0_1 = self.uniform()
        random_idx = int(uniform_0_1 * len(lst))
        return lst[random_idx]

def create_val_pairs(data_df: pd.DataFrame,
                     padding_size: int = 15,
                     min_group_size: int = 2):
    data_df = data_df[['id_left', 'id_right', 'label']]
    group_sizes = data_df.groupby('id_left').size()
    groups_to_use = group_sizes[group_sizes >= min_group_size].index.tolist()

    all_ids = set(data_df['id_left']).union(set(data_df['id_right']))
    out_pairs_df = data_df.copy()
    out_pairs_df = out_pairs_df[out_pairs_df.id_left.isin(groups_to_use)]
    out_pairs_df['label'] += 1

    out_pairs_list = out_pairs_df.values.tolist()
    random_generator = BufferedRandomValuesGenerator(len(data_df))

    for group_id in group_sizes[group_sizes < padding_size].index:
        df = data_df[data_df.id_left == group_id]
        group_ids = set(df.id_right.values).union({group_id})
        num_pad_items = padding_size - len(df)
        unrelevant_ids = list(all_ids - group_ids)
        # pad_sample = np.random.choice(, num_pad_items, replace=False).tolist()
        # out_pairs_df.append({
        #     'id_left': [group_id] * num_pad_items,
        #     'id_right': pad_sample,
        #     'label': [0] * num_pad_items
        # }, ignore_index=True)
        for i in range(num_pad_items):
            right_id = random_generator.choice(unrelevant_ids)
            out_pairs_list.append([group_id, right_id, 0])
    return out_pairs_list

%lprun create_val_pairs(test_df)

In [19]:
%prun dataset.create_val_pairs(test_df)

In [8]:
set(test_df.id_right.values)

{524290,
 524292,
 262149,
 131076,
 262159,
 393234,
 131096,
 393245,
 30,
 524322,
 36,
 38,
 42,
 262190,
 262195,
 51,
 393271,
 262200,
 393273,
 57,
 393275,
 60,
 131131,
 64,
 131137,
 131141,
 131142,
 70,
 524357,
 131155,
 262227,
 262233,
 131167,
 97,
 131178,
 108,
 131181,
 524399,
 116,
 118,
 134,
 393355,
 131214,
 393358,
 144,
 146,
 262290,
 148,
 393362,
 147,
 262295,
 131224,
 131228,
 158,
 159,
 160,
 262307,
 262309,
 166,
 170,
 172,
 173,
 174,
 177,
 178,
 180,
 393397,
 131255,
 184,
 262329,
 131264,
 393408,
 524480,
 131272,
 202,
 131276,
 210,
 524499,
 214,
 218,
 393440,
 131297,
 224,
 393445,
 131301,
 393455,
 262387,
 244,
 262391,
 262398,
 262401,
 131330,
 264,
 266,
 131340,
 131341,
 272,
 393489,
 274,
 524565,
 262426,
 393499,
 282,
 131355,
 524575,
 131360,
 524583,
 524585,
 300,
 524594,
 262455,
 393529,
 262459,
 316,
 320,
 524608,
 131394,
 131400,
 333,
 262479,
 337,
 338,
 131423,
 262497,
 262501,
 524646,
 524649,
 362,
 5

In [12]:
test_df[['id_left', 'id_right', 'label']][:50].values.tolist()

[[303345, 303346, 0],
 [69383, 380476, 0],
 [266948, 175089, 1],
 [29298, 29299, 0],
 [314169, 314170, 0],
 [449673, 311332, 1],
 [10927, 229257, 1],
 [299859, 299860, 1],
 [443182, 514615, 0],
 [8780, 8781, 0],
 [470800, 470801, 0],
 [101280, 101281, 0],
 [7806, 7807, 1],
 [388153, 486522, 0],
 [67240, 67241, 0],
 [445065, 75098, 1],
 [178157, 4251, 1],
 [91188, 370547, 0],
 [456281, 456282, 0],
 [432498, 432499, 1],
 [513878, 513879, 0],
 [13248, 40432, 1],
 [22080, 190016, 0],
 [268938, 268939, 0],
 [142117, 142118, 0],
 [171754, 171755, 0],
 [167503, 286338, 0],
 [483088, 389865, 0],
 [78012, 76103, 0],
 [482176, 482177, 0],
 [153771, 382192, 1],
 [504625, 504626, 0],
 [67214, 208516, 0],
 [39242, 54576, 1],
 [59140, 59141, 0],
 [173861, 387069, 0],
 [475855, 475856, 0],
 [77423, 463097, 0],
 [443562, 443563, 1],
 [220972, 220973, 0],
 [295914, 442083, 0],
 [141947, 247963, 0],
 [10160, 45457, 1],
 [372960, 372961, 0],
 [458630, 335031, 0],
 [498328, 498329, 1],
 [19892, 12515, 1],

In [131]:
train_dataloader = dataset.make_train_dataloader(train_df, vocabulary_list)

train_dataloader

In [132]:
test_dataloader = dataset.make_test_dataloader(test_df, vocabulary_list)

test_dataloader

# Model training and inference

## Few words about ndcg metrics that we are using


In [96]:
# TODO add model evaluation to the metrics lib

## Code for the model evaluation

In [86]:
import torch

In [138]:
{1}.union({2})

{1, 2}

In [134]:
from metrics import ndcg_k

def evaluate(model: torch.nn.Module = None, data: torch.utils.data.DataLoader = None) -> float:
    labels_and_groups = data.dataset.samples_list
    labels_and_groups = pd.DataFrame(labels_and_groups, columns=['left_id', 'right_id', 'rel'])

    pred = [model.predict(batch).detach().numpy() for batch, _ in data]
    pred = np.concatenate(pred, axis=0)
    labels_and_groups['pred'] = pred

    ndcg_list = [ndcg_k(df.rel.values, df.pred.values) for _, df in labels_and_groups.groupby('left_id')]
    mean_ndcg = float(np.mean(ndcg_list))
    return mean_ndcg


In [139]:
import numpy as np

In [141]:
{1,2,3}-{3}

{1, 2}

## Training loop

In [96]:
n_epochs = 20
learning_rate = 0.01
from tqdm.notebook import tqdm

In [ ]:
# TODO: add model evaluation into the code

val_dataset is created
val_dataloader is created


In [135]:
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss = torch.nn.BCELoss()

for epoch in tqdm(range(n_epochs)):
    model.train()
    for batch_idx, (left_batch, right_batch, y_true) in enumerate(train_dataloader):
#         left_batch = {k: v for k, v in left_batch.items()}
#         right_batch = {k: v for k, v in right_batch.items()}
        opt.zero_grad()
        y_pred = model.forward(left_batch, right_batch)
        query_loss = loss(y_pred, y_true)
        query_loss.backward()
        opt.step()
    ndcg_score = evaluate(model, test_dataloader)
    print(f"Epoch {epoch}. Test ndcg: {ndcg_score:.3f}")

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0. Test ndcg: 0.513
Epoch 1. Test ndcg: 0.800
Epoch 2. Test ndcg: 0.808
Epoch 3. Test ndcg: 0.802
Epoch 4. Test ndcg: 0.816
Epoch 5. Test ndcg: 0.836
Epoch 6. Test ndcg: 0.862
Epoch 7. Test ndcg: 0.882
Epoch 8. Test ndcg: 0.898
Epoch 9. Test ndcg: 0.911
Epoch 10. Test ndcg: 0.917
Epoch 11. Test ndcg: 0.921
Epoch 12. Test ndcg: 0.925
Epoch 13. Test ndcg: 0.928
Epoch 14. Test ndcg: 0.930
Epoch 15. Test ndcg: 0.932
Epoch 16. Test ndcg: 0.934
Epoch 17. Test ndcg: 0.937
Epoch 18. Test ndcg: 0.938
Epoch 19. Test ndcg: 0.940


# Final words

## How to infer

## How to save / load the model

In [ ]:
with open('resources/mlp_weights01.pkl', 'wb') as f:
    torch.save(solution.model.mlp.state_dict(), f)

with open('resources/embeddings.pkl', 'wb') as f:
    torch.save(solution.model.embeddings.state_dict(), f)

## Effective search on big corpora

In [1]:
import numpy as np

import string
import re
import pandas as pd
import datetime 
import torch
import solution as s
import importlib
importlib.reload(s)
import aux
import dataset

In [30]:
importlib.reload(aux)
importlib.reload(dataset)


<module 'dataset' from '/Users/pavel/src/other/search-ranking/training/dataset.py'>

/Users/pavel/miniconda3/envs/ds_py3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  """Entry point for launching an IPython kernel.
/Users/pavel/miniconda3/envs/ds_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  


KeyboardInterrupt: 

In [6]:
embeddings = aux.Embeddings(glove_path,[train_df, test_df])

		create_glove_emb_from_file
		glove_dict is read from file
		glove_dict is embeddings_list is created
		glove_dict is embeddings_list is initialized
		glove_dict is embeddings matrix is created


In [5]:
# dev_pairs_for_ndcg = dataset.create_val_pairs(test_df)

KNRM embeddings is created
KNRM kernels is created
KNRM mlp is created


In [81]:
import solution as s
import importlib
importlib.reload(s)

solution = s.Solution(
    model=model,
    train_factory = train_factory,
    test_factory = test_factory
)

val_dataset is created
val_dataloader is created


In [82]:
%%time
solution.train(2)
solution.evaluate()

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 31.1 s, sys: 1.39 s, total: 32.5 s
Wall time: 23.3 s


0.6274717234859845

In [43]:
solution.train(2)
solution.evaluate()

  0%|          | 0/2 [00:00<?, ?it/s]

0.6531143801280398

In [62]:
solution.evaluate()

0.6531143801280398

In [47]:
labels_and_groups = solution.test_dataloader.dataset.index_pairs_or_triplets

In [48]:
labels_and_groups = pd.DataFrame(labels_and_groups, columns=['left_id', 'right_id', 'rel'])
labels_and_groups

,left_id,right_id,rel
0,100141,75743,2
1,100141,100142,2
2,100141,518798,0
3,100141,268627,0
4,100141,405675,0
...,...,...,...
31090,99774,100343,0
31091,99774,277792,0
31092,99774,109829,0
31093,99774,363096,0


In [61]:
all_preds = []
for batch, _ in solution.test_dataloader:
    preds = model.predict(batch)
    preds = preds.detach().numpy()
    all_preds.append(preds)
all_preds = np.concatenate(all_preds, axis=0)
labels_and_groups['preds'] = all_preds


In [60]:
labels_and_groups

,left_id,right_id,rel,preds
0,100141,75743,2,1.748668
1,100141,100142,2,1.748600
2,100141,518798,0,1.648066
3,100141,268627,0,1.382531
4,100141,405675,0,1.442735
...,...,...,...,...
31090,99774,100343,0,1.313921
31091,99774,277792,0,1.599022
31092,99774,109829,0,1.375366
31093,99774,363096,0,1.439974


In [ ]:
ndcgs = []
for cur_id in labels_and_groups.left_id.unique():
    cur_df = labels_and_groups[labels_and_groups.left_id == cur_id]
    ndcg = ndcg_k(cur_df.rel.values.reshape(-1), cur_df.preds.values.reshape(-1))
    if np.isnan(ndcg):
        ndcgs.append(0)
    else:
        ndcgs.append(ndcg)
return np.mean(ndcgs)


In [64]:
for _, df in labels_and_groups.groupby('left_id'):
    ndcg = ndcg_k(df.rel.values.reshape(-1), df.preds.values.reshape(-1))
    if np.isnan(ndcg):
        ndcgs.append(0)
    else:
        ndcgs.append(ndcg)
return np.mean(ndcgs)


In [67]:
import metrics

In [75]:
[metrics.ndcg_k(df.rel.values, df.preds.values)
 for _, df in labels_and_groups.groupby('left_id')]

[1.0,
 0.9197207891481876,
 1.0,
 0.430624116386567,
 0.5249810332008934,
 1.0,
 0.9005687248376788,
 0.5706417189553201,
 0.17723928678404774,
 0.5049211793563422,
 0.6934264036172708,
 0.6240505200038378,
 0.6509209298071326,
 0.3065735963827292,
 0.17723928678404774,
 1.0,
 0.7903864795495061,
 0.6131471927654584,
 0.9197207891481876,
 0.23719771276929622,
 1.0,
 0.8772153153380493,
 0.8175295903539446,
 0.5706417189553201,
 0.411833840435435,
 0.6673071725340496,
 1.0,
 0.44158011035778233,
 0.6797310500037657,
 1.0,
 0.41443699955334395,
 0.5847876649948586,
 1.0,
 0.8628540832451718,
 0.9757880548549218,
 0.1815417925373527,
 0.5109559939712154,
 0.4415801103577823,
 0.5706417189553201,
 0.6934264036172708,
 0.86034433104172,
 0.6885288809404666,
 0.6934264036172708,
 1.0,
 0.6131471927654584,
 0.5360595666833466,
 0.6131471927654584,
 0.4684505201610771,
 0.46845052016107697,
 0.17723928678404774,
 0.39780880120575696,
 0.45749452618986164,
 1.0,
 0.8315546295836226,
 0.81752959

In [71]:
labels_and_groups.rel.values

array([2, 2, 0, ..., 0, 0, 0])

In [78]:
float(np.mean([1,2,3]))

2.0

In [ ]:
with open('resources/mlp_weights01.pkl', 'wb') as f:
    torch.save(solution.model.mlp.state_dict(), f)

with open('resources/embeddings.pkl', 'wb') as f:
    torch.save(solution.model.embeddings.state_dict(), f)

In [ ]:
import json
with open('resources/vocabulary.json', 'w') as f:
    json.dump(solution.vocab, f)

# Model save / load checking

In [ ]:
test_df = pd.read_csv('../problem1_5/quora/dev.tsv', sep='\t')
test_df = test_df.drop('id',axis=1)
test_df = test_df.rename(columns={'qid1':'doc_id','qid2': 'query_id', 'question1':'doc','question2':'query'})
test_df['doc_len'] = test_df.doc.str.len()
test_df['query_len'] = test_df['query'].str.len()
test_df = test_df.sort_values('query_len')
test_df

In [ ]:
def tokenize_text(text, max_len=30, oov=solution.vocab['OOV']):
    tokens = solution.simple_preproc(text)[:max_len]
    token_ids = [solution.vocab.get(token, oov)
                 for token in tokens]
    pad_len = max_len - len(token_ids)
    if pad_len > 0:
        token_ids = token_ids + [0] * pad_len
    return token_ids

In [ ]:
test_df['score'] = test_df.apply(calc_score, axis=1)

In [ ]:
df = test_df.sort_values('score').reset_index(drop=True).reset_index()
df["bucket"] =  df['index'] // 1100
df.groupby('bucket').is_duplicate.mean()

In [ ]:
import ranking_model as r
import tokenizer as t
import importlib
importlib.reload(t)
importlib.reload(r)

tokenizer = t.Tokenizer('resources/vocabulary.json', 30)
ranking_model = r.RankingModel("resources/embeddings.pkl", 'resources/mlp_weights01.pkl')

In [ ]:
def calc_score(row):
    doc_tokens = torch.tensor(tokenize_text(row['doc'])).reshape(1,-1)
    query_tokens = torch.tensor(tokenize_text(row['query'])).reshape(1,-1)
    score = solution.model.predict({'query':query_tokens, 'document':doc_tokens})
    return score.item()

def calc_score2(row):
    doc_tokens = tokenizer.tokenize_text(row['doc'])
    query_tokens = tokenizer.tokenize_text(row['query'])
    
    score = ranking_model.predict(query_tokens, doc_tokens)
    return score

print(calc_score(row))
print(calc_score2(row))

In [ ]:
query_tokens = torch.tensor(tokenize_text(row['query'])).reshape(1,-1)
doc_tokens = torch.tensor(tokenize_text(row['doc'])).reshape(1,-1)

In [ ]:
emb_query1 = solution.model.embeddings(query_tokens)
emb_query2 = ranking_model.model.embeddings(query_tokens)

In [ ]:
matching_matrix1 = solution.model._get_matching_matrix(query_tokens, doc_tokens)
matching_matrix2 = ranking_model.model._get_matching_matrix(query_tokens, doc_tokens)

In [ ]:
kernels_repr1 = solution.model._apply_kernels(matching_matrix1)
kernels_repr2 = ranking_model.model._apply_kernels(matching_matrix2)
kernels_repr1, kernels_repr2

In [ ]:
mlp_out1 = solution.model.mlp(kernels_repr1)
mlp_out2 = ranking_model.model.mlp(kernels_repr2)
mlp_out1, mlp_out2

In [ ]:
test_df['score2'] = test_df.apply(calc_score2, axis=1)

In [ ]:
df = test_df.sort_values('score2').reset_index(drop=True).reset_index()
df['bucket'] = df['index'] // 1100
df.groupby('bucket').is_duplicate.mean()

In [ ]:
import torch